## Steps
 * get 10000 links
 * get data from links
 * see where all required data is saved
 * clean data (empty rows, numerical values, etc.)


In [3]:
import bs4
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json
import re
import lxml.html
import time
import random
from random import randint
import logging
import collections
from time import gmtime, strftime

import re
from tabulate import tabulate
import os

date = strftime("%Y-%m-%d")

import selenium

# The selenium.webdriver module provides all the implementations of WebDriver
# Currently supported are Firefox, Chrome, IE and Remote. The `Keys` class provides keys on
# the keyboard such as RETURN, F1, ALT etc.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

## Get the initial link and its data

Using the page in the url we loop through the pages untill we have as many links as we need (currently 100). This takes a while, due to needing time to fully load the page before we can get the data (30 links per page though, maybe something I can do with asynchronous stuff so we don't have to wait as long(just rand time between 0 and 2 sec maybe))

In [4]:
links = []

page = 1

while len(links) < 100:
    url = f"https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page={page}&orderBy=newest"

    driver = webdriver.Firefox()
    driver.implicitly_wait(5)
    driver.get(url)

    #python_button = driver.find_elements_by_xpath('//div[@data-reactid="269"]')[0]
    #python_button.click()

    # And then we use Beautiful soup
    soup = BeautifulSoup(driver.page_source)

    driver.close()

    for list_item in soup.find_all("li", attrs={"class": "search-results__item"}):
        for link in list_item.find_all("a", attrs={"class": "card__title-link"}):
            links.append(link.get("href"))
    
    page += 1

## Print the links

Just to get a bit of an overview, can we maybe spot a pattern? (Sadly, no)

In [5]:
#for link in links:
    #print(link)

## Get the data from one of the links

We now make a soup from just 1 of the links so we can start looking at the data and which data we need. Later we will replace this with a loop so we can start building soups from the different links. Currently using just a regular get request, hopefully we can get all the necessary data with this, not really looking forward to having to start and close drivers for every link.

In [66]:
r = requests.get(links[0])
soup = BeautifulSoup(r.content, "lxml")

Now we'll take our soup and look for the relevant data. Since ImmoWeb fills it's site using data from a database we simply need to locate this data, parse it and store the relevant parts for easier access.

In [73]:
div_with_script = soup.find("div", attrs={"class": "classified"})
script_text = div_with_script.script.text.split("= ", 1)[1]
json_data = json.loads(script_text.rstrip()[:-1])

property_data = json_data["property"]
price_data = json_data["price"]
for key in relevant_data.keys():
    print(key)
    print(relevant_data[key])
    print("\n")

type
HOUSE


subtype
PAVILION


title
None


description
None


name
None


isHolidayProperty
None


bedroomCount
3


bedrooms
[]


bathroomCount
None


bathrooms
[]


location
{'country': 'Belgium', 'region': 'Wallonie', 'province': 'Hainaut', 'district': 'Tournai', 'locality': 'Leuze-En-Hainaut', 'postalCode': '7900', 'street': "à 10' de Tournai", 'number': None, 'box': None, 'propertyName': None, 'floor': None, 'latitude': 50.5985901, 'longitude': 3.6173008, 'distance': None, 'approximated': None, 'regionCode': 'WALLONIE', 'type': 'RESIDENTIAL', 'hasSeaView': None, 'pointsOfInterest': [{'type': 'SCHOOL', 'distance': 0}, {'type': 'SHOPS', 'distance': 0}, {'type': 'TRANSPORT', 'distance': 0}], 'placeName': None}


netHabitableSurface
125


roomCount
None


monthlyCosts
None


attic
None


hasAttic
None


basement
None


hasBasement
None


hasDressingRoom
None


diningRoom
None


hasDiningRoom
None


building
{'annexCount': None, 'condition': 'AS_NEW', 'constructionYear': 2003, 'facade

In [13]:
script_text = soup.find_all("script")
script_text = soup.find_all("script")[1].text.split("= ", 1)[1]

json_data = json.loads(script_text[:script_text.find(";")])
print(json_data[0]["classified"])

{'id': '9778032', 'type': 'house', 'subtype': 'pavilion', 'price': '180000', 'transactionType': 'for sale', 'zip': '7900', 'visualisationOption': 'l', 'kitchen': {'type': ''}, 'building': {'constructionYear': '2003', 'condition': 'as new'}, 'energy': {'heatingType': 'fueloil'}, 'certificates': {'primaryEnergyConsumptionLevel': '311'}, 'bedroom': {'count': '3'}, 'land': {'surface': '2104'}, 'atticExists': '', 'basementExists': '', 'outdoor': {'garden': {'surface': ''}, 'terrace': {'exists': ''}}, 'specificities': {'SME': {'office': {'exists': ''}}}, 'wellnessEquipment': {'hasSwimmingPool': ''}, 'parking': {'parkingSpaceCount': {'indoor': '', 'outdoor': ''}}, 'condition': {'isNewlyBuilt': 'false'}}


In [75]:
# Locality
# <span class="classified__information--address-row"> 
# -->                                       Rue des Francais 1,41 
#                                <!----></span>
print("location")
print(property_data["location"])

# Type of property (House/apartment)
# Located in link as indicated below, maybe start fitering before getting data, so we can split the poroperty types
# https://www.immoweb.be/en/classified/ [[apartment]] /for-sale/ans/4430/9777582?searchId=621c956edcf6b 

print("type: ")
print(property_data["type"])

# Subtype of property (Bungalow, Chalet, Mansion, ...)
# I don't directly see this in the link, but there is an option to filter this on the main page so there should be a way to get this info

print("subtype: ")
print(property_data["subtype"])

# Price
# <p class="classified__price"><span aria-hidden="true">
# -->                                   €204,900
#                                </span> <span class="sr-only">204900€</span></p>

print("price: ")
print(price_data["mainValue"])

# Type of sale (Exclusion of life sales)
# Not sure what is asked here specifically

print("transactionType: ")
print(price_data["type"]) #?

# Number of rooms
# Is this number of bedrooms? casue if so:
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">
#                                    Bedrooms
#                                </th> <td class="classified-table__data">
# -->                                   3
#                                </td></tr>

print("number of bedrooms: ")
print(property_data["bedroomCount"])

#Area
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">
#                                    Living area
#                                </th> <td class="classified-table__data">
# -->                                   90
#
#                                    <span class="abbreviation"><span aria-hidden="true">
# -->                                           m²                                        </span> <span class="sr-only">
#                                            square meters
#                                        </span></span></td></tr>

print("Area: ")
print(property_data["netHabitableSurface"])

# Fully equipped kitchen (Yes/No)
# Not sure but maybe
# <tr class="classified-table__row"><th scope="row" class="classified-table__header">Kitchen type</th> <td class="classified-table__data">
# -->                                   Installed
#                                </td></tr>

print("kitchen: ")
print(property_data['kitchen']) #get rest of data from here

#Furnished (Yes/No)
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">Furnished</th> <td class="classified-table__data">
# -->                                   No
#                                </td></tr>
print("furnished: ")
print(json_data["transaction"]["sale"]["isFurnished"])

#Open fire (Yes/No)
# Not sure but maybe
#<tr class="classified-table__row"><th scope="row" class="classified-table__header">Heating type</th> <td class="classified-table__data">
# -->                               Gas
#                            </td></tr>

#<tr class="classified-table__row"><th scope="row" class="classified-table__header">
#                                    How many fireplaces?
#                            </th> <td class="classified-table__data">
# -->                                   2
#                            </td></tr>

print("fireplaces: ")
print(property_data["fireplaceExists"])

#Terrace (Yes/No)
    #If yes: Area

print("terrace: ")
print(property_data["hasTerrace"]) 
print(property_data["terraceOrientation"])
    
#Garden (Yes/No)
    #If yes: Area

print("garden: ")
print(property_data["hasGarden"])
print(property_data["gardenOrientation"])
    
#Surface of the land

print("surface: ")
print(property_data["land"]["surface"])

#Surface area of the plot of land

print("surface: ")
print(property_data['constructionPermit']['totalBuildableGroundFloorSurface'])

#Number of facades
# <tr class="classified-table__row"><th scope="row" class="classified-table__header">
#                                    Number of frontages
#                                </th> <td class="classified-table__data">
# -->                                   4
#                                </td></tr>

print("facades: ")
print(property_data["building"]["facadeCount"])

#Swimming pool (Yes/No)

print("pool: ")
print(property_data["hasSwimmingPool"]) #get rest of data from this

#State of the building (New, to be renovated, ...)

print("condition: ")
print(property_data['building']['condition']) #get rest of data from this

location
{'country': 'Belgium', 'region': 'Wallonie', 'province': 'Hainaut', 'district': 'Tournai', 'locality': 'Leuze-En-Hainaut', 'postalCode': '7900', 'street': "à 10' de Tournai", 'number': None, 'box': None, 'propertyName': None, 'floor': None, 'latitude': 50.5985901, 'longitude': 3.6173008, 'distance': None, 'approximated': None, 'regionCode': 'WALLONIE', 'type': 'RESIDENTIAL', 'hasSeaView': None, 'pointsOfInterest': [{'type': 'SCHOOL', 'distance': 0}, {'type': 'SHOPS', 'distance': 0}, {'type': 'TRANSPORT', 'distance': 0}], 'placeName': None}
type: 
HOUSE
subtype: 
PAVILION
price: 
180000
transactionType: 
annuity_monthly_amount
number of bedrooms: 
3
Area: 
125
kitchen: 
None
furnished: 
None
fireplaces: 
False
terrace: 
None
None
garden: 
None
None
surface: 
2104
surface: 
0
facades: 
4
pool: 
None
condition: 
AS_NEW
